In [13]:
path = "./cifar/"
import numpy as np
import pickle
from PIL import Image
import matplotlib.pyplot as plt

def extractImagesAndLabels(path, file):
    f = open(path+file, 'rb')
    dicte = pickle.load(f,encoding='bytes')
    images = dicte[b'data']
    Matrix=[]
    for i in images:
        ingle_img_reshaped = np.transpose(np.reshape(i,(3, 32,32)), (1,2,0))
        #x=np.dot(ingle_img_reshaped[...,:3], [0.299, 0.587, 0.114])
        x= ingle_img_reshaped.flatten()
        Matrix.append(x)
    #images = np.transpose(images, (1,2,0))
   
    Matrix = np.array(Matrix)
    labels = dicte[b'labels']
    labels = np.array(labels)
    #print labels.shape
    return Matrix, labels
 
#     labels = dict['labels']

def extractCategories(path, file):
    f = open(path+file, 'rb')
    dict = pickle.load(f,encoding='bytes')
    return dict[b'label_names']

def saveCifarImage(array, path, file):
    # array is 3x32x32. cv2 needs 32x32x3
    array = array.asnumpy().transpose(1,2,0)
    # array is RGB. cv2 needs BGR
    array = cv2.cvtColor(array, cv2.COLOR_RGB2BGR)
    # save to PNG file
    return cv2.imwrite(path+file+".png", array)


Train_data, Train_labels = extractImagesAndLabels(path, "data_batch_1")
Train_data_2, Train_labels_2 = extractImagesAndLabels(path,"data_batch_2")
Train_data_3, Train_labels_3 = extractImagesAndLabels(path, "data_batch_3")
Train_data_3, Train_labels_3 = extractImagesAndLabels(path, "data_batch_3")
Train_data_4, Train_labels_4 = extractImagesAndLabels(path, "data_batch_3")
Train_data_5, Train_labels_5 = extractImagesAndLabels(path, "data_batch_3")
Train_data_t = np.concatenate((Train_data, Train_data_2,Train_data_3,Train_data_4,Train_data_5), axis=0)
Train_labels_t = np.concatenate((Train_labels,Train_labels_2,Train_labels_3,Train_labels_4,Train_labels_5), axis=0)
Test_data, Test_labels = extractImagesAndLabels(path, "test_batch")

In [15]:
import torch
# X = torch.FloatTensor(X.train.shape[0], X_train.shape[1])

    
# D = torch.FloatTensor(X_train.shape[1],k)

# R = torch.FLoatTensor(k,X_train.shape[0])
D_weights = torch.FloatTensor(10, 10).normal_()
R_weights = torch.FloatTensor(10, 10).normal_()


# image shape





    
# def load():
#     files = os.path.join(DATA_DIR,'stl10_binary')
#     cross_val_indices = []
#     with open(os.path.join(files, "fold_indices.txt"), "r") as ins:
#         for line in ins:
#             cross_val_indices.append([int(x) for x in line.rstrip().split(' ')])
#     X_train = read_all_images(os.path.join(files,"train_X.bin")).astype('float32')
#     Y_train = read_labels(os.path.join(files,"train_y.bin"))
#     X_test = read_all_images(os.path.join(files,"test_X.bin")).astype('float32')
#     Y_test = read_labels(os.path.join(files,"test_y.bin"))
#     X_train /= 255.
#     X_test /= 255.
#     return (X_train, Y_train), (X_test, Y_test), cross_val_indices

# def unlabelled_data():
#     files = os.path.join(DATA_DIR,'stl10_binary')
#     unlabelled_x = read_all_images(os.path.join(files,"unlabeled_X.bin")).astype('float32')
#     unlabelled_x /= 255.
#     return unlabelled_x


In [28]:

import torch
import os
import struct
import matplotlib.pyplot as plt
import numpy as np

from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as f
import torch.optim as optim
import copy

import matplotlib
matplotlib.use('Agg')


# from load_data import load, unlabelled_data

# Don't hog GPU
# config = tf.ConfigProto()
# config.gpu_options.allow_growth=True
# sess = tf.Session(config=config)
import torch.optim as optim

def dictionary_learning(X_train, lambda_val = 1, k = 128, learning_rate=1):
    X = (torch.FloatTensor(X_train)
    D_weights = Variable(torch.FloatTensor(X_train.shape[1], k).normal_(),requires_grad=True)
    R_train = Variable(torch.FloatTensor(k, X_train.shape[0]).normal_(),requires_grad=True)
#     minimization_term = torch.norm(X - torch.transpose(torch.mm(D_weights, R_weights)),2)
#     regularization_term = lambda_val*(torch.nonzero(R_weights.data).size(0))

    # Normal MSE loss for autoencoder:
#     loss = minimization_term + regularization_term
    D_iter = iter(D_weights)
    R_iter = iter(R_train)
    optimizer = optim.SGD(D_iter, lr= 0.01)
    optimizer_alpha = optim.SGD(R_iter, lr = 0.01)    
    num_epochs = 100

    errors = []
    for i in range(num_epochs):
        minimization_term = torch.norm(X - torch.transpose(torch.mm(D_weights, R_train),0,1),2)
        regularization_term = lambda_val*(torch.nonzero(R_train.data).size(0))
        loss = minimization_term+regularization_term
        loss /= X_train.shape[0]
        # ===================backward====================
        optimizer_alpha.zero_grad()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        
        optimizer_alpha.step()
        print("Training loss (MSE) : %.4f" % (loss))

    plt.clf()
    plt.plot(1 + np.arange(num_epochs), errors, label='Training loss')
    plt.title('Training loss v/s Epochs')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    plt.show()
    plt.savefig('dict_training.png')

    return Dw, R_train.T



X_train,Y_train,X_test, Y_test = Train_data_t, Train_labels_t, Test_data, Test_labels



# def find_reprs(X_train, D_train, lambda_val = 1, k=128, learning_rate=1):
#     X = tf.placeholder("float", X_train.shape)

#     D = tf.placeholder("float", D_train.shape)
#     R = tf.placeholder("float", (k, X_train.shape[0]))
    
#     R_weights = tf.Variable(tf.random_normal([k, X_train.shape[0]]))
    
#     minimization_term = tf.norm(X - tf.transpose(tf.matmul(D, R_weights)), ord='fro', axis=(0,1))
#     regularization_term = lambda_val * tf.cast(tf.count_nonzero(R_weights), tf.float32)

#     # Normal MSE loss for autoencoder:
#     loss = minimization_term + regularization_term
#     loss /= X_train.shape[0]
#     optimizer = tf.train.AdadeltaOptimizer(learning_rate)
    
#     R_new = optimizer.minimize(loss, var_list=[R_weights])
    
#     init = tf.global_variables_initializer()
#     sess.run(init)
    
#     num_epochs = 50

#     # Train model
#     R_train = None
#     for i in range(num_epochs):

#         # Update R
#         _, l, R_train = sess.run([R_new, loss, R_weights],feed_dict={X: X_train, D: D_train})
#         print("Representation finding loss (MSE) : %.4f" % (l))

#     return R_train.T

D, features_train = dictionary_learning(X_train)
# features_test = find_reprs(test_flat_X, D, 0.01, 100, 100)
#features_test = np.linalg.pinv(test_flat_X, D.T)



def eucledian_predict(X_tr, Y_tr, X_te, Y_te):
    correct = 0
    for i, test in enumerate(X_te):
        distances = []
        for point in X_tr:
            distances.append(np.linalg.norm(test - point))
        label = Y_tr[np.argmin(distances)]
        if np.argmax(label) == np.argmax(Y_te[i]):
            correct += 1
        accuracy  = correct / (1.0 * Y_te.shape[0])
    return accuracy


# print("Dictionary-learning features based L2 accuracy:",eucledian_prediction(features_train, Y_train, features_test, Y_test))

# if not os.path.exists("atoms"):
#         os.makedirs("atoms")

# for i, atom in enumerate(D.T):
#         vis_atom = (atom.reshape((96, 96, 3)) * 255).astype('uint8')
#         plt.clf()
#         plt.imshow(vis_atom)
#         plt.savefig("atoms/" + str(i+1) + "atom.png")

/home/mohit1_aml/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/home/mohit1_aml/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/mohit1_aml/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/mohit1_aml/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/mohit1_aml/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/mohit1_aml/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.

ValueError: can't optimize a non-leaf Tensor